In [ ]:
from flask import Flask
from flask import request
import json
import requests
import re


app = Flask(__name__)

regex = re.compile('[!,/\().%]')

@app.route("/analyze", methods=['GET', 'POST'])
def analyze():
    """
    Analyzer provide stemm, tokenize and lang detect workflow
    
    Receive: list of lists of textes
    Return: list of lists of stemmed tokens for each document + languages
    """
    if request.method=='POST':
        data = request.json
        
        textes = data["textes"]
        textes = [regex.sub(' ',text) for text in textes]
        r_lang = requests.post('http://127.0.0.1:13567/lang_detect', json={'textes' : textes}).text
        langs = json.loads(r_lang)['langs']
        
        r_stemmed = requests.post('http://127.0.0.1:13566/stemmer', json={'textes' : textes, 'langs': langs}).text
        stemmed_docs = json.loads(r_stemmed)['stemmed']

        return json.dumps({'status':'ok',
                           'langs': langs,
                           'stemmed_docs': stemmed_docs})
    else:
        return "post json {textes: 'your text here'}"
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13571)

 * Running on http://0.0.0.0:13571/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2018 18:52:32] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2018 18:52:49] "POST /analyze HTTP/1.1" 200 -
